In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook as wb

In [2]:
GAC_SEs = pd.read_csv("/Users/rtv-lpt-129/Downloads/PEAL - Standards Evaluation - June_GAC (1)(1).csv", encoding='latin1', low_memory = False)

In [3]:
GAC_SEs.head(1)

,SubmissionDate,starttime,endtime,text_audit,ev_date,district_0,enumerator,cluster_0,village_0,ev_month,...,liquid_manure_nursery,organic_pesticides_nursery,nursery_watered,crop_varieties_planted,village_instance,final_instance,instanceID,instanceName,formdef_version,KEY
0,"Jul 12, 2024 5:44:49 PM","Jul 12, 2024 8:28:53 AM","Jul 12, 2024 1:46:27 PM",https://rtvuganda.surveycto.com/view/submissio...,12/07/2024,Kagadi,Kasangaki Micheal,Buhumuliro,Kasozi,23,...,NaN,NaN,NaN,NaN,Kasozi,Household SE:Benadi Byaruhanga,uuid:8ceb220c-8592-4729-a334-d7f96e0b15a7,Household SE:Benadi Byaruhanga,2407040804,uuid:8ceb220c-8592-4729-a334-d7f96e0b15a7


In [4]:
HHLevel = GAC_SEs[GAC_SEs['standards_report'] == 'Household'].copy()

In [5]:
HHLevel['Water control Practice'] = np.where(HHLevel['water_mgt_methods_num'] > 0, 1,0)
HHLevel['Soil Management Practice'] = np.where(HHLevel['soil_mgt_methods_num'] > 1, 1,0)
HHLevel['KAP Livestock'] = np.where(HHLevel['kap_livestock_num'] > 3, 1,0)
HHLevel['KAP Agriculture'] = np.where(HHLevel['kap_farm_practices_num'] > 3, 1,0)
HHLevel['3 Composts'] = np.where(HHLevel['composts_num'] > 2, 1,0)
HHLevel['Vegetable gardening Practices'] = np.where(HHLevel['vegetable_farm_mode_num'] > 0, 1,0)

In [6]:
conditions = [
    (HHLevel['hh_age'] < 31), 
    (HHLevel['hh_age'] > 30) & (HHLevel['hh_gender'] == 1),
    (HHLevel['hh_age'] > 30) & (HHLevel['hh_gender'] == 2)
]

# Define the corresponding outputs
choices = ['Youthheaded', 'Maleheaded', 'Femaleheaded']

# Create the 'headship' column
HHLevel['headship'] = np.select(conditions, choices)

In [10]:
conditions = [
    (HHLevel['age_1'] < 31), 
    (HHLevel['age_1'] > 30) & (HHLevel['hh_sex'] == 'Male'),
    (HHLevel['age_1'] > 30) & (HHLevel['hh_sex'] == 'Female')
]

# Define the corresponding outputs
choices = ['Youthheaded', 'Maleheaded', 'Femaleheaded']

# Create the 'headship' column
HHLevel['headship'] = np.select(conditions, choices)

In [11]:
HHLevel['headship'].head()

0    Youthheaded
1     Maleheaded
2     Maleheaded
3     Maleheaded
4     Maleheaded
Name: headship, dtype: object

In [12]:
HHLevel[['Water control Practice', 'Soil Management Practice', 
     'Vegetable gardening Practices', 'KAP Livestock', 'KAP Agriculture', '3 Composts']].head(1)

,Water control Practice,Soil Management Practice,Vegetable gardening Practices,KAP Livestock,KAP Agriculture,3 Composts
0,1,1,0,0,1,1


In [13]:
HHLevel['Compost Compliance'] =  HHLevel[['3 Composts', 'composts_standard', 'forms_non_bio',
                                  'composts_free_runoff', 'composts_manure_usuage']].mean(axis = 1)

In [14]:
HHLevel['Drying Method Num'] = HHLevel[['drying_method_1','drying_method_2',
                                    'drying_method_3']].sum(axis = 1)

In [15]:
HHLevel['Drying Method'] = np.where(HHLevel['Drying Method Num'] > 0,1,0)

In [16]:
HHLevel['Post harvest handling'] = HHLevel[['Drying Method', 'sorting_grading_done', 
                                     'post_harvest_sys_present']].mean(axis = 1)

In [17]:
HHLevel['Agric Compliance'] = HHLevel[['Compost Compliance', 'Post harvest handling', 'Water control Practice',
                               'Soil Management Practice', 'Vegetable gardening Practices', 
                               'hh_apply_lq_manure', 'hh_organic_pesticides']].mean(axis = 1)

In [18]:
WASH = ['latrine_present', 'latrine_cover_present', 'latrine_floor_sealed',
                              'latrine_door_present', 'latrine_walls_sealed', 'latrine_roofleak_proof',
                              'latrine_environ_free_debris', 'latri_environ_free_feaces', 'tippy_tap_present',
                              'tippy_fill_water', 'soap_ash_present', 'tippy_tap_wet_area', 'cloth_hangline',
                              'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present',
                              'bath_drainage_present', 'bathroom_clean', 'compound_clean', 'double_dishrack',
                              'non_bio_waste_properly_sorted']

In [19]:
HHLevel[WASH] = HHLevel[WASH].apply(pd.to_numeric, errors='coerce')

In [20]:
HHLevel['WASH Compliance'] = HHLevel[WASH].mean(axis = 1)

In [21]:
Livestock = ['gs_pen_constr', 'gs_pen_constr', 'gs_droppings_composted', 
             'pg_sty_constr', 'pg_sty_clean', 'pg_droppings_composted']

In [22]:
HHLevel['Livestock Compliance'] = HHLevel[Livestock].mean(axis = 1)

In [23]:
HHLevel['Overall Compliance'] = HHLevel[['Agric Compliance', 'WASH Compliance', 
                                 'Livestock Compliance', 'vlsa_membership']].mean(axis = 1)

In [24]:
HHLevel['NPS Classified'] = np.where(HHLevel['overall_nps'] < 7, 'Detractor', 
                                 np.where(HHLevel['overall_nps'] > 8, 'Promoter','Passive'))

In [25]:
required_columns = ['Detractor', 'Passive', 'Promoter']


# Function to ensure columns exist and initialize if missing
def ensure_columns_exist(df, columns):
    for col in columns:
        if col not in df.columns:
            df[col] = 0

# Ensure required columns in Village_NPS
ensure_columns_exist(Village_NPS, required_columns)

# Ensure required columns in Cluster_NPS
ensure_columns_exist(Cluster_NPS, required_columns)

# Create the 'Total' column for Village_NPS
Village_NPS['Total'] = Village_NPS['Detractor'] + Village_NPS['Passive'] + Village_NPS['Promoter']

# Create the 'Total' column for Cluster_NPS
Cluster_NPS['Total'] = Cluster_NPS['Detractor'] + Cluster_NPS['Passive'] + Cluster_NPS['Promoter']

# Calculate percentages for Village_NPS
Village_NPS['Detractors(%)'] = (Village_NPS['Detractor'] / Village_NPS['Total']) * 100
Village_NPS['Passives(%)'] = (Village_NPS['Passive'] / Village_NPS['Total']) * 100
Village_NPS['Promoters(%)'] = (Village_NPS['Promoter'] / Village_NPS['Total']) * 100

Village_NPS['Overall NPS (%)'] = Village_NPS['Promoters(%)'] - Village_NPS['Detractors(%)']

# Calculate percentages for Cluster_NPS
Cluster_NPS['Detractors(%)'] = (Cluster_NPS['Detractor'] / Cluster_NPS['Total']) * 100
Cluster_NPS['Passives(%)'] = (Cluster_NPS['Passive'] / Cluster_NPS['Total']) * 100
Cluster_NPS['Promoters(%)'] = (Cluster_NPS['Promoter'] / Cluster_NPS['Total']) * 100

Cluster_NPS ['Overall NPS (%)'] = Cluster_NPS ['Promoters(%)'] - Cluster_NPS ['Detractors(%)']

Cluster_NPS  = Cluster_NPS .reset_index()[
    ['district_0', 'cluster_0', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]
Village_NPS = Village_NPS.reset_index()[
    ['district_0', 'cluster_0', 'village_0', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]

Village_NPS.to_excel('GACVillage_NPS.xlsx', index=False)
Cluster_NPS .to_excel('GACNPSCluster.xlsx', index=False)

NameError: name 'Village_NPS' is not defined

In [26]:
Village_NPS = HHLevel.groupby(['district_0', 'cluster_0', 'village_0'])['NPS Classified'].value_counts().unstack(fill_value=0)

Village_NPS = Village_NPS.reindex(columns=['Detractor', 'Passive', 'Promoter'], fill_value=0)

Village_NPS['Total'] = Village_NPS.sum(axis=1)

Village_NPS['Detractors(%)'] = (Village_NPS['Detractor'] / Village_NPS['Total']) * 100
Village_NPS['Passives(%)'] = (Village_NPS['Passive'] / Village_NPS['Total']) * 100
Village_NPS['Promoters(%)'] = (Village_NPS['Promoter'] / Village_NPS['Total']) * 100

Village_NPS['Overall NPS (%)'] = Village_NPS['Promoters(%)'] - Village_NPS['Detractors(%)']

Village_NPS = Village_NPS.reset_index()[
    ['district_0', 'cluster_0', 'village_0', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]
Village_NPS.to_excel('GACVillage_NPS.xlsx', index=False)

In [27]:
HHLevel['VSLACommitte'] = np.where(HHLevel['vlts_committee_score'] < 7, 'Detractor', 
                                 np.where(HHLevel['overall_nps'] > 8, 'Promoter','Passive'))

In [28]:
VLT_NPS = HHLevel.groupby(['district_0', 'cluster_0', 'village_0'])['VSLACommitte'].value_counts().unstack(fill_value=0)

VLT_NPS = VLT_NPS.reindex(columns=['Detractor', 'Passive', 'Promoter'], fill_value=0)

VLT_NPS['Total'] = VLT_NPS.sum(axis=1)

VLT_NPS['Detractors(%)'] = (VLT_NPS['Detractor'] / VLT_NPS['Total']) * 100
VLT_NPS['Passives(%)'] = (VLT_NPS['Passive'] / VLT_NPS['Total']) * 100
VLT_NPS['Promoters(%)'] = (VLT_NPS['Promoter'] / VLT_NPS['Total']) * 100

VLT_NPS['Overall NPS (%)'] = VLT_NPS['Promoters(%)'] - VLT_NPS['Detractors(%)']

VLT_NPS = VLT_NPS.reset_index()[
    ['district_0', 'cluster_0', 'village_0', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]
VLT_NPS.to_excel('GACVSLA_NPS.xlsx', index=False)

In [29]:
HHLevel['Agric Champions'] = np.where(HHLevel['agric_champion_score'] < 7, 'Detractor', 
                                 np.where(HHLevel['overall_nps'] > 8, 'Promoter','Passive'))

In [30]:
Agric_NPS = HHLevel.groupby(['district_0', 'cluster_0', 'village_0'])['Agric Champions'].value_counts().unstack(fill_value=0)

Agric_NPS = Agric_NPS.reindex(columns=['Detractor', 'Passive', 'Promoter'], fill_value=0)

Agric_NPS['Total'] = Agric_NPS.sum(axis=1)

Agric_NPS['Detractors(%)'] = (Agric_NPS['Detractor'] / Agric_NPS['Total']) * 100
Agric_NPS['Passives(%)'] = (Agric_NPS['Passive'] / Agric_NPS['Total']) * 100
Agric_NPS['Promoters(%)'] = (Agric_NPS['Promoter'] / Agric_NPS['Total']) * 100

Agric_NPS['Overall NPS (%)'] = Agric_NPS['Promoters(%)'] - Agric_NPS['Detractors(%)']

Agric_NPS = Agric_NPS.reset_index()[
    ['district_0', 'cluster_0', 'village_0', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]
Agric_NPS.to_excel('GACAgric_NPS.xlsx', index=False)

In [31]:
HHLevel['WASH Champions'] = np.where(HHLevel['wash_champion_score'] < 7, 'Detractor', 
                                 np.where(HHLevel['overall_nps'] > 8, 'Promoter','Passive'))

In [32]:
WASH_NPS = HHLevel.groupby(['district_0', 'cluster_0', 'village_0'])['WASH Champions'].value_counts().unstack(fill_value=0)

WASH_NPS = WASH_NPS.reindex(columns=['Detractor', 'Passive', 'Promoter'], fill_value=0)

WASH_NPS['Total'] = WASH_NPS.sum(axis=1)

WASH_NPS['Detractors(%)'] = (WASH_NPS['Detractor'] / WASH_NPS['Total']) * 100
WASH_NPS['Passives(%)'] = (WASH_NPS['Passive'] / WASH_NPS['Total']) * 100
WASH_NPS['Promoters(%)'] = (WASH_NPS['Promoter'] / WASH_NPS['Total']) * 100

WASH_NPS['Overall NPS (%)'] = WASH_NPS['Promoters(%)'] - WASH_NPS['Detractors(%)']

WASH_NPS = WASH_NPS.reset_index()[
    ['district_0', 'cluster_0', 'village_0', 'Detractors(%)', 'Passives(%)', 'Promoters(%)', 'Overall NPS (%)']
]
WASH_NPS.to_excel('GACWASH_NPS.xlsx', index=False)

In [33]:
Village_Scores = HHLevel.groupby(['district_0',  'cluster_0',  'village_0', 'ev_month'])[['Overall Compliance', 'Agric Compliance', 'WASH Compliance', 
                            'Livestock Compliance', 'vlsa_membership', 'Compost Compliance', 
                            '3 Composts', 'composts_standard', 'forms_non_bio', 'composts_free_runoff', 
                            'composts_manure_usuage', 'Drying Method', 'drying_method_1','drying_method_2',
                            'drying_method_3', 'drying_method_4', 'sorting_grading_done', 
                            'post_harvest_sys_present', 'Post harvest handling', 'Water control Practice',
                            'Soil Management Practice', 'Vegetable gardening Practices', 'hh_apply_lq_manure', 
                            'hh_organic_pesticides', 'bbw_mgt', 'latrine_present', 'latrine_cover_present', 'latrine_floor_sealed',
                              'latrine_door_present', 'latrine_walls_sealed', 'latrine_roofleak_proof',
                              'latrine_environ_free_debris', 'latri_environ_free_feaces', 'tippy_tap_present',
                              'tippy_fill_water', 'soap_ash_present', 'tippy_tap_wet_area', 'cloth_hangline',
                              'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present',
                              'bath_drainage_present', 'bathroom_clean', 'compound_clean', 'double_dishrack',
                              'non_bio_waste_properly_sorted', 'livestock_present_1', 'livestock_present_2',  'gs_pen_constr', 'gs_pen_clean', 'gs_droppings_composted', 
                            'livestock_present_3', 'pg_sty_constr', 'pg_sty_clean', 'pg_droppings_composted', 'KAP Livestock', 
                            'KAP Agriculture']].mean(numeric_only = True).reset_index()
Village_Scores = Village_Scores.sort_values(
    by=['district_0', 'cluster_0', 'Overall Compliance'], 
    ascending=[True, True, False]
)

Village_Scores.to_excel('GACVillage_Scores.xlsx', index = False)

In [34]:
Cluster_Scores = HHLevel.groupby(['district_0',  'cluster_0', 'ev_month'])[['Overall Compliance', 'Agric Compliance', 'WASH Compliance', 
                            'Livestock Compliance', 'vlsa_membership', 'Compost Compliance', 
                            '3 Composts', 'composts_standard', 'forms_non_bio', 'composts_free_runoff', 
                            'composts_manure_usuage', 'Drying Method', 'drying_method_1','drying_method_2',
                            'drying_method_3', 'drying_method_4', 'sorting_grading_done', 
                            'post_harvest_sys_present', 'Post harvest handling', 'Water control Practice',
                            'Soil Management Practice', 'Vegetable gardening Practices', 'hh_apply_lq_manure', 
                            'hh_organic_pesticides', 'bbw_mgt', 'latrine_present', 'latrine_cover_present', 'latrine_floor_sealed',
                              'latrine_door_present', 'latrine_walls_sealed', 'latrine_roofleak_proof',
                              'latrine_environ_free_debris', 'latri_environ_free_feaces', 'tippy_tap_present',
                              'tippy_fill_water', 'soap_ash_present', 'tippy_tap_wet_area', 'cloth_hangline',
                              'kitchen_present', 'kitchen_ventilated', 'kitchen_clean', 'bathroom_present',
                              'bath_drainage_present', 'bathroom_clean', 'compound_clean', 'double_dishrack',
                              'non_bio_waste_properly_sorted', 'livestock_present_1', 'livestock_present_2',  'gs_pen_constr', 'gs_pen_clean', 'gs_droppings_composted', 
                            'livestock_present_3', 'pg_sty_constr', 'pg_sty_clean', 'pg_droppings_composted', 'KAP Livestock', 
                            'KAP Agriculture']].mean(numeric_only = True).reset_index()

Cluster_Scores.to_excel('GACCluster_Scores.xlsx', index = False)

In [35]:
HHLevel['bbw_mgt'] = HHLevel['bbw_mgt'].replace('True',1)

In [37]:
headhsipcluster = HHLevel.groupby(['district_0',  'cluster_0', 'headship'])[['KAP Livestock', 
                            'KAP Agriculture']].mean(numeric_only = True).reset_index()

headhsipcluster.to_excel('Headship.xlsx', index = False)

In [38]:
headhsipvillage = HHLevel.groupby(['district_0',  'cluster_0', 'village_0', 'headship'])[['KAP Livestock', 
                            'KAP Agriculture']].mean(numeric_only = True).reset_index()

headhsipvillage.to_excel('Headship1.xlsx', index = False)

In [39]:
headhsipcluster1 = headhsipcluster.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)
headhsipcluster1.to_excel('HeadshipC.xlsx', index = False)
headhsipvillage1 = headhsipvillage.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)
headhsipvillage1.to_excel('HeadshipV.xlsx', index = False)

/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_58772/517156281.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  headhsipcluster1 = headhsipcluster.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)
/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_58772/517156281.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  headhsipvillage1 = headhsipvillage.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)


In [30]:
print(HHLevel['bbw_mgt'])

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2822   NaN
2823   NaN
2824   NaN
2825   NaN
2826   NaN
Name: bbw_mgt, Length: 2652, dtype: float64


In [31]:
BBW = HHLevel.groupby(['district_0',  'cluster_0', 'ev_month'])[[ 'bbw_mgt', 'drying_method_1','drying_method_2',
                            'drying_method_3']].mean(numeric_only = True).reset_index()

In [32]:
print(BBW)

   district_0       cluster_0  ev_month   bbw_mgt  drying_method_1  \
0      Kagadi      Buhumuliro        23  0.633333         0.977273   
1      Kagadi        Kaitemba        12  0.272727         0.977778   
2      Kagadi  Kihuura_Kagadi        12  0.333333         0.847619   
3      Kagadi       Kikomagwa         6  0.090909         0.977528   
4      Kagadi          Kitema        12  0.422414         0.956938   
5      Kagadi         Kitooro        12  0.407407         0.985294   
6      Kagadi       Kyarwakya         6  0.275862         0.986755   
7      Kagadi          Mbogwa         6  0.000000         0.966667   
8      Kagadi      Nyabutanzi         6  0.269841         0.992537   
9      Kagadi      Nyakashema        23  0.627451         0.991667   
10     Kagadi        Rusekere         6  0.437500         0.966443   
11     Kagadi         Ruzaire         6  0.580000         1.000000   
12     Kagadi        Yorodani        23  0.295082         1.000000   
13   Kyenjojo      K

In [184]:
Cluster_ScoresA = Cluster_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)
Cluster_ScoresA.to_excel('GACCluster_ScoresA.xlsx', index = False)
Village_ScoresA = Village_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)
Village_ScoresA.to_excel('GACVillage_ScoresA.xlsx', index = False)

/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_70772/3746447759.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Cluster_ScoresA = Cluster_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)
/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_70772/3746447759.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  Village_ScoresA = Village_Scores.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)


In [67]:
def determine_conclusion(score):
    if score < 70:
        return "Off-Track - Issue of Missing Program Targets"
    elif 70 <= score <= 79:
        return "At Risk - Below Program Targets with Major Areas to Address"
    elif 80 <= score <= 89:
        return "On Track - Near Program Targets with Minor Areas to Address"
    else:
        return "On Track - Meet or Exceed Program Targets"

# Apply the function to the "Overall score" column to create a new "Conclusions" column
Village_ScoresA['Overall Compliance Comment'] = Village_ScoresA['Overall Compliance'].apply(determine_conclusion)
Village_ScoresA['Agric Compliance Comment'] = Village_ScoresA['Agric Compliance'].apply(determine_conclusion)
Village_ScoresA['WASH Compliance Comment'] = Village_ScoresA['WASH Compliance'].apply(determine_conclusion)
Village_ScoresA['Livestock Compliance Comment'] = Village_ScoresA['Livestock Compliance'].apply(determine_conclusion)
Village_ScoresA['VSLA Participation Comment'] = Village_ScoresA['vlsa_membership'].apply(determine_conclusion)

# Create a new DataFrame with the "Overall score" and "Conclusions" columns
Conclusions = Village_ScoresA[['district_0',  'cluster_0', 'village_0', 'ev_month','Overall Compliance Comment',
                              'Agric Compliance Comment', 'WASH Compliance Comment', 'Livestock Compliance Comment',
                              'VSLA Participation Comment']]

Conclusions.to_excel('GACConclusions.xlsx', index = False)

In [71]:
VLevel = GAC_SEs[GAC_SEs['standards_report'] == 'Village'].copy()

In [72]:
VLevel['VSLAs_num'] = VLevel[['vsla_type1_1', 'vsla_type1_2', 'vsla_type1_3']].sum(axis = 1)

In [80]:
VLevel_SEs = VLevel.groupby(['district_0',  'cluster_0',  'village_0', 'ev_month'])[['water_sources_num', 'wuc1', 'wuc1_fetch_records', 'wuc1_contribution_records', 'water_flow1', 
                                                                         'source_sanitary_standards1', 'source_structural_standards1', 'source_demarcation_1', 'pooling_clear_1', 
                                                                         'time_spent_reduce1', 'VSLAs_num', 'mems_active_joint', 'contr_documentation_joint', 'keep_record_members_joint', 
                                                                         'keep_record_loans_expense_joint', 'meet_regular_joint', 'om_balance_costs_joint', 'vsla_goals_documented_joint', 
                                                                         'active_committee_func_joint', 'committee_positions_filled_joint', 'loan_repayments_tracked_join', 'loaning_intrest_joint',
                                                                         'mems_active_youth', 'contr_documentation_youth', 'keep_record_members_youth', 
                                                                         'keep_record_loans_expense_youth', 'meet_regular_youth', 'om_balance_costs_youth', 'vsla_goals_documented_youth', 
                                                                         'active_committee_func_youth', 'committee_positions_filled_youth', 'loan_repayments_tracked_youth', 'loaning_intrest_youth',
                                                                         'mems_active_women', 'contr_documentation_women', 'keep_record_members_women', 
                                                                         'keep_record_loans_expense_women', 'meet_regular_women', 'om_balance_costs_women', 'vsla_goals_documented_women', 
                                                                         'active_committee_func_women', 'committee_positions_filled_women', 'loan_repayments_tracked_women', 'loaning_intrest_women',
                                                                         'demo_cared', 'farm_pract_seen', 'organic_manure_demo', 'organic_pesticides_demo', 'liquid_manure_demo',
                                                                         'nursery_shade_constr', 'farm_pract_seen_nurs', 'liquid_manure_nursery', 
                                                                         'organic_pesticides_nursery', 'nursery_watered']].mean(numeric_only = True).reset_index()

VLevel_SEs.fillna("-", inplace=True)

VLevel_SEsA = VLevel_SEs.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)

/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_70772/1241415352.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  VLevel_SEs.fillna("-", inplace=True)
/var/folders/8v/g75zp59111n27nbw3_c9ypxh0000gn/T/ipykernel_70772/1241415352.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  VLevel_SEsA = VLevel_SEs.applymap(lambda x: x * 100 if isinstance(x, (int, float)) else x)


In [81]:
VLevel_SEsA.to_excel('GACVLevel_SEs.xlsx', index = False)

In [82]:
Comments = HHLevel[['district_0',  'cluster_0',  'village_0', 'ev_month', 'hh_comment',  'evaluators_comment']]

In [84]:
Comments.to_excel('GACComments.xlsx', index = False)

In [8]:
head = HHLevel[['age_1', 'hh_sex', 'headship']]

In [9]:
head.to_excel('head.xlsx', index = False)